In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -10..10, -10..10,  -10..10
box = E × x × y
using GLMakie
using LinearAlgebra, LinearSolve,Statistics

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [2]:
@inbounds function Res(u, p, t)
    du1 = -u[2]-u[3]
    du2 = u[1]+p[1]*u[2]
    du3 = p[2]*u[1]-p[3]*u[3]+u[1]*u[3]
    return SVector(du1, du2, du3)
end
@inbounds function jac_res(u, p, t)
    SMatrix{3, 3}(0.0, 1.0, p[2]+u[3],
                -1.0, p[1], 0.0,
                -1.0, 0.0, -p[3]+u[1])
end

jac_res (generic function with 1 method)

In [13]:
function condition(u, t, integrator)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf) - ϵ_box
end
affect!(integrator) = terminate!(integrator)
function get_norm_αs(u)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf), linsolve
end
function color_alpha(alpha)
    if alpha > 0.0
        return "red"
    else
        return "blue"
    end
end;

In [142]:
b = 0.3;  c = 3.458; a = 0.4916
t = 1000.0; tspan = [0.0, t]
p = [a, b, c]
u0 = [0.0, 0.0, 0.0];

In [164]:
ds = CoupledODEs(Res, u0, p)
fp, _, _ = fixedpoints(ds, box, jac_res)

ϵ_box = 1e-3
Ju0 = jac_res(fp[1], p, 0);
fp_ = fp[1]

eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors

v1 = real(eigen_vectors[:, 1])
v2 = real(eigen_vectors[:, 2])
v3 = imag(eigen_vectors[:, 3])

v1 = normalize(v1, 2)
v1t = transpose(v1)

v2 = normalize(v2, 2)
v2t = transpose(v2)

v3 = normalize(v3, 2)
v3t = transpose(v3)

A = transpose([v1t; v2t; v3t])
A = Matrix(A)

3×3 Matrix{Float64}:
  0.264692   0.960091    0.0
 -0.068448  -0.269859   -0.999772
  0.961901   0.0734972  -0.0213429

In [165]:
total_number_points = 50
number_points_on_side = Int((total_number_points) / 1)

dots_right_side = zeros(number_points_on_side, length(u0))
array_α_vec_right = zeros(number_points_on_side, length(u0))

dots_left_side = zeros(number_points_on_side, length(u0))
array_α_vec_left = zeros(number_points_on_side, length(u0))

dots_up_side = zeros(number_points_on_side, length(u0))
array_α_vec_up = zeros(number_points_on_side, length(u0))

dots_down_side = zeros(number_points_on_side, length(u0))
array_α_vec_down = zeros(number_points_on_side, length(u0));

In [166]:
# case α1 = 0; α2 = ϵ_box; α3 = [-ϵ_box; ϵ_box]
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = ϵ_box
    α3 = - ϵ_box + 2 * ϵ_box * n / N
    α_vec = [α1, α2, α3]
    point = fp_ - A * α_vec
    
    dots_right_side[n, :] = point
    _, α_vec = get_norm_αs( dots_right_side[n, :] )
    array_α_vec_right[n, :] = α_vec
    n+=1
end

# case α1 = 0; α2 = -ϵ_box; α3 = [-ϵ_box; ϵ_box]
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = -ϵ_box
    α3 = - ϵ_box + 2 * ϵ_box * n / N
    α_vec = [α1, α2, α3]
    point = fp_ - A * α_vec
    
    dots_left_side[n, :] = point
    _, α_vec = get_norm_αs( dots_left_side[n, :] )
    array_α_vec_left[n, :] = α_vec
    n+=1
end

# case α1 = 0; α2 = [-ϵ_box; ϵ_box]; α3 = ϵ_box
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = - ϵ_box + 2 * ϵ_box * n / N
    α3 = ϵ_box
    α_vec = [α1, α2, α3]
    point = fp_ - A * α_vec
    
    dots_up_side[n, :] = point
    _, α_vec = get_norm_αs( dots_up_side[n, :] )
    array_α_vec_up[n, :] = α_vec
    n+=1
end

# case α1 = 0; α2 = [-ϵ_box; ϵ_box]; α3 = -ϵ_box
n = 1
N = total_number_points
while n <= number_points_on_side

    α1 = 0.0
    α2 = - ϵ_box + 2 * ϵ_box * n / N
    α3 = -ϵ_box
    α_vec = [α1, α2, α3]
    point = fp_ - A * α_vec
    
    dots_down_side[n, :] = point
    _, α_vec = get_norm_αs( dots_down_side[n, :] )
    array_α_vec_down[n, :] = α_vec
    n+=1
end;

In [187]:
index = 21
cb = ContinuousCallback(condition, nothing, affect!)
u0 = SA[dots_right_side[index, 1], dots_right_side[index, 2], dots_right_side[index, 3]]

prob = ODEProblem(Res, u0, tspan, p)
sol = solve(prob, alg = Vern9(), abstol = 1e-14, reltol = 1e-14, callback = cb)

retcode: Terminated
Interpolation: specialized 9th order lazy interpolation
t: 4-element Vector{Float64}:
 0.0
 0.03801195986720886
 0.07040914977085999
 0.07040914977085999
u: 4-element Vector{SVector{3, Float64}}:
 [-0.000960090900479025, 0.00010989496621080494, -7.69120814540523e-5]
 [-0.0009606704563583019, 7.511438322372939e-5, -7.769976248112245e-5]
 [-0.0009600895863647313, 4.495483063513492e-5, -7.829362332957826e-5]
 [-0.0009600895863647313, 4.495483063513492e-5, -7.829362332957826e-5]

In [188]:
norm_, alpha_ = get_norm_αs(sol[end])

(0.0010000000000000002, [4.9646981798181785e-9, -0.0010000000000000002, 0.0002249545914335952])

In [189]:
width, height = 600, 500
markersize = 5.0
color = :black
xlabel = L"x"; ylabel = L"y"; zlabel = L"E"
xlabelsize = 25; ylabelsize = 25; zlabelsize = 25

tstart, tend = 1, length(sol)
f = Figure(resolution = (width, height))
#ax = Axis3(f[1, 1], xlabel = xlabel, ylabel = ylabel, zlabel = zlabel, xlabelsize = xlabelsize, ylabelsize = ylabelsize, zlabelsize = zlabelsize)

ax = LScene(f[1, 1], show_axis = true)

lines!(sol[2, tstart:tend], sol[3, tstart:tend], sol[1, tstart:tend], color = :deeppink, linewidth = 1.0)

scatter!(ax, dots_right_side[:, 2], dots_right_side[:, 3], dots_right_side[:, 1],  markersize = markersize, color = color)
scatter!(ax, dots_left_side[:, 2], dots_left_side[:, 3], dots_left_side[:, 1],  markersize = markersize, color = color)
scatter!(ax, dots_up_side[:, 2], dots_up_side[:, 3], dots_up_side[:, 1],  markersize = markersize, color = color)
scatter!(ax, dots_down_side[:, 2], dots_down_side[:, 3], dots_down_side[:, 1],  markersize = markersize, color = color)

scatter!(ax, fp_[2], fp_[3], fp_[1],  markersize = markersize, color = :blue)
display(GLMakie.Screen(), f)

GLMakie.Screen(...)

In [131]:
width, height = 600, 500
markersize = 5.0
color = color_alpha(alpha_[1])
xlabel = L"α_2"; ylabel = L"α_3"
xlabelsize = 25; ylabelsize = 25

f = Figure(resolution = (width, height))
ax = Axis(f[1, 1], xlabel = xlabel, ylabel = ylabel, xlabelsize = xlabelsize, ylabelsize = ylabelsize)

scatter!(ax, alpha_[2], alpha_[3], markersize = markersize, color = color)


scatter!(ax, 0.0, 0.0, markersize = markersize, color = :green)
display(GLMakie.Screen(), f)

GLMakie.Screen(...)

In [19]:
width, height = 600, 500
markersize = 5.0
color = :black
xlabel = L"x"; ylabel = L"y"; zlabel = L"z"
xlabelsize = 25; ylabelsize = 25; zlabelsize = 25

f = Figure(resolution = (width, height))
ax = Axis3(f[1, 1], xlabel = xlabel, ylabel = ylabel, zlabel = zlabel, xlabelsize = xlabelsize, ylabelsize = ylabelsize, zlabelsize = zlabelsize)

scatter!(ax, dots_right_side[:, 2], dots_right_side[:, 3], dots_right_side[:, 1],  markersize = markersize, color = color)
scatter!(ax, dots_left_side[:, 2], dots_left_side[:, 3], dots_left_side[:, 1],  markersize = markersize, color = color)
scatter!(ax, dots_up_side[:, 2], dots_up_side[:, 3], dots_up_side[:, 1],  markersize = markersize, color = color)
scatter!(ax, dots_down_side[:, 2], dots_down_side[:, 3], dots_down_side[:, 1],  markersize = markersize, color = color)

scatter!(ax, fp_[2], fp_[3], fp_[1],  markersize = markersize, color = :blue)
display(GLMakie.Screen(), f)

GLMakie.Screen(...)